## Types of parallelism

### Embarissingly Parallel

### Vectorised operators

Fortran:
sse/avx intrinsics
optimisation flags

Python:
numpy

### Shared Memory

Fortran:
openmp

Python:
Dask

### Distributed

Fortran:
MPI

Python:
Multiprocessing
Dask Distributed

## Design Patterns

## Common Gotchas

### Race Conditions

### Locks